# Data 가져오기

In [1]:
import pandas as pd
import numpy as np

In [2]:
datapath = "./data/winequalityN.csv"

data_sets = pd.read_csv(datapath, delimiter = ',')

data_sets.dropna(inplace = True)

x_data = data_sets.drop(columns = ["type", "quality"],axis = 1)
y_data = data_sets["quality"]

x_data = x_data.dropna(axis = 0)
y_data = y_data.dropna(axis = 0)

In [3]:
print(x_data.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    6463.000000       6463.000000  6463.000000     6463.000000   
mean        7.217755          0.339589     0.318758        5.443958   
std         1.297913          0.164639     0.145252        4.756852   
min         3.800000          0.080000     0.000000        0.600000   
25%         6.400000          0.230000     0.250000        1.800000   
50%         7.000000          0.290000     0.310000        3.000000   
75%         7.700000          0.400000     0.390000        8.100000   
max        15.900000          1.580000     1.660000       65.800000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  6463.000000          6463.000000           6463.000000  6463.000000   
mean      0.056056            30.516865            115.694492     0.994698   
std       0.035076            17.758815             56.526736     0.003001   
min       0.009000             1.000000         

In [4]:
# def norm(df,key):
#     c = df[key]
#     df_min = c.min()
#     df_max = c.max()
# #     print("key=> ",key,"min",df_min,"max",df_max)
#     df[key] = (c-df_min)/(df_max-df_min)

# x_columns = x_data.columns.tolist()

# for i in x_columns:
#     norm(x_data, i)

In [5]:
x_data = x_data.values.tolist()
y_data = y_data.values.tolist()

In [6]:
x_data = np.array(x_data)
y_data = np.array(y_data)

print(x_data.shape)
print(y_data.shape)

(6463, 11)
(6463,)


# SVM

In [16]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC

In [113]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data,test_size = 0.2, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2,random_state=0)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(4136, 11)
(4136,)
(1034, 11)
(1034,)
(1293, 11)
(1293,)


# 모델 선택

In [101]:
def model_selection(x_data, y_data):
    kernels = ['linear','poly','rbf','sigmoid']
    svm_model = []
    scores = []
    
    for i in kernels:
        temp_model = make_pipeline(StandardScaler(), svm.SVR(C = 1.0, epsilon = 0.1, kernel = i))
        scores.append(cross_val_score(temp_model, x_data, y_data).mean())
    
    for i,j in enumerate(scores):
        print("model : {}, score : {}".format(kernels[i], j))
        
model_selection(x_data,y_data)

model : linear, score : 0.029835686378
model : poly, score : -45.0375244424
model : rbf, score : 0.118551953206
model : sigmoid, score : -2049.52950192


# 모델 선언

In [114]:
def get_predictions(model, x_data):
    predictions = model.predict(x_data)
    predictions = np.around(predictions)
    
    return predictions

def model(x_train, y_train):
    svm_model = make_pipeline(StandardScaler(), svm.SVR(C = 100, epsilon = 0.1, kernel = 'rbf'))

    svm_model.fit(x_train, y_train)
    
    predictions = get_predictions(svm_model, x_train)
    
    return svm_model, predictions

m1, p = model(x_train, y_train)

# Vaildation 검증

In [115]:
score = m1.score(x_val, y_val)

print("SVM 의 교차검증 점수 : {}".format(score))

SVM 의 교차검증 점수 : 0.229015998535


# Vaildation train

In [116]:
m1.fit(x_val, y_val)

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svr', SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False))])

# 모델 평가

In [117]:
print("모델 평가 : {}".format(m1.score(x_train, y_train)))
print("Train set 정확도: {:.2f}".format(accuracy_score(y_train, p)))

모델 평가 : -0.136246313065
Train set 정확도: 0.81


# Test

In [118]:
print("Test set 정확도: {:.2f}".format(accuracy_score(y_test, get_predictions(m1, x_test))))

Test set 정확도: 0.45


https://wdprogrammer.tistory.com/29

https://m.blog.naver.com/PostView.nhn?blogId=owl6615&logNo=221459656642&proxyReferer=https:%2F%2Fwww.google.com%2F